In [ ]:
import paddle
import os
from paddlenlp.transformers import SkepForSequenceClassification, SkepTokenizer

# load skep
model = SkepForSequenceClassification.from_pretrained(pretrained_model_name_or_path="skep_ernie_2.0_large_en", num_classes=2)
# tokenizer loaded
tokenizer = SkepTokenizer.from_pretrained(pretrained_model_name_or_path="skep_ernie_2.0_large_en")

In [ ]:
# choose model directory
params_path = './skep_ckpt//model_2200/model_state.pdparams'
if params_path and os.path.isfile(params_path):
    # load model
    state_dict = paddle.load(params_path)
    model.set_dict(state_dict)
    print("Loaded parameters from %s" % params_path)
else:
    print("Model not found")

In [ ]:
index = 0
model.eval()

import pandas as pd
covid_data = pd.read_csv('./project-data/covid19_tweets.csv')

with open('./results/covid19_tweets_sentiment.csv', 'w+', encoding='utf-8') as f:
    f.write('label\n')

    for i in range(len(covid_data)):
        text = covid_data['text'][i]
        inputs = tokenizer(text)
        inputs = {k:paddle.to_tensor([v]) for (k, v) in inputs.items()}
        output = model(**inputs)
                
        pred_labels = [item.argmax().item() for item in output]
        f.write(str(pred_labels[0]) + '\n')

In [ ]:
import json
from utils import clean_text
from tqdm import tqdm
model.eval()

with open('./project-data/covid.data.txt', 'r') as f:
    instance_lines = f.readlines()

instances = []
for line in instance_lines:
    temp = line.strip('\n').split(',')
    for item in temp:
        if os.path.exists('./project-data/task2-tweet-objects/' + str(item)+'.json'):
            instances.append(item)

with open('./results/task2-tweets.csv', 'w+', encoding='utf-8') as f:   
    f.write('Id,Label\n')            
    for item in tqdm(instances):
        f2 = open('./project-data/task2-tweet-objects/' + item + '.json', 'r', encoding='utf-8')
        content = json.load(f2)
        text = clean_text(content['text'])
        f2.close()

        inputs = tokenizer(text)
        inputs = {k:paddle.to_tensor([v]) for (k, v) in inputs.items()}
        output = model(**inputs)
                
        pred_labels = [item.argmax().item() for item in output]
        f.write(item + ',' + str(pred_labels[0]) + '\n')